In [ ]:
!pip install --upgrade google-cloud-storage


Defining Project ID from Google Cloud project id and selecting the Region

In [ ]:
PROJECT_ID="[your project ID]"
REGION="us-central1"

Authenticating Google Cloud login and signup

In [ ]:
! gcloud auth login

Configuring Google Cloud and setting project

In [ ]:
! gcloud config set project "[your project ID]"

Authenticating Google colab with google cloud

In [ ]:
from google.colab import auth
auth.authenticate_user()


Creating New Bucket on Google Cloud

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

For Exisiting Google cloud Bucket

In [ ]:
#BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"

Importing files form local drive eg CSV format and converting it into JSONL format and upload it to gcs bucket

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

filename = next(iter(uploaded))
df = pd.read_csv(filename)
df.head()

jsonl_str  = ""

# Create a string for JSONL format
for i, row in df.iterrows():
    #print(row['Inline_text'], row['Label'])
    desc = row['Inline_text'].replace("'",'').replace('"',"").replace("\n","")
    jsonl_str += '''{"systemInstruction": {"role": "system","parts": [{"text": "You are a Classifier responsible to classify into CTS,CTS Campus,Classroom Support,Print Team (External)."}]}, "contents": [{"role": "user","parts": [{"text": 'DESC'}]},{"role": "model","parts": [{"text": 'LABEL'}]}]}'''.replace("DESC", desc).replace("LABEL", row['Label'])
    jsonl_str += "\n"

output_path = "sample_data/updated_email.jsonl"
with open(output_path, 'w') as file:
    file.write(jsonl_str)
bucket_name = "[your Bucket name]"# paste here
!gcloud storage cp {output_path} gs://{bucket_name}/

Fintuning AI model with JSONL file

In [ ]:

import time

import vertexai
from vertexai.tuning import sft

vertexai.init(project=PROJECT_ID, location="us-central1")

sft_tuning_job = sft.train(
    source_model="gemini-2.0-flash-001",
    # 1.5 and 2.0 models use the same JSONL format
    train_dataset="your Bucket name"#Paste here,
)

# Polling for job completion
while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(f'model name: {sft_tuning_job.tuned_model_name}')
print(f'endpoint: {sft_tuning_job.tuned_model_endpoint_name}')
print(f'experiment: {sft_tuning_job.experiment}')
print(f'tuning job: {sft_tuning_job}')
# Example response:
# projects/123456789012/locations/us-central1/models/1234567890@1
# projects/123456789012/locations/us-central1/endpoints/123456789012345
# <google.cloud.aiplatform.metadata.experiment_resources.Experiment object at 0x7b5b4ae07af0>

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob
INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/3231630305152466944?project=69210017139


model name: projects/69210017139/locations/us-central1/models/1739237179630026752@1
endpoint: projects/69210017139/locations/us-central1/endpoints/8848013060783210496
experiment: <google.cloud.aiplatform.metadata.experiment_resources.Experiment object at 0x7b29e5d1e3d0>
tuning job: <vertexai.tuning._supervised_tuning.SupervisedTuningJob object at 0x7b29e5d1e450> 
resource name: projects/69210017139/locations/us-central1/tuningJobs/3231630305152466944


For Viewing Job lists

In [ ]:
!gcloud ai hp-tuning-jobs list --region=us-central1



View a list of tuning jobs

In [ ]:
import vertexai
from vertexai.tuning import sft

vertexai.init(project=PROJECT_ID, location="us-central1")

responses = sft.SupervisedTuningJob.list()

for response in responses:
    print(response)
# Example response:
# <vertexai.tuning._supervised_tuning.SupervisedTuningJob object at 0x7c85287b2680>
# resource name: projects/12345678/locations/us-central1/tuningJobs/123456789012345

Get details of a tuning job

In [ ]:
import vertexai
from vertexai.tuning import sft

LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

tuning_job_id = "7223697540862967808"
response = sft.SupervisedTuningJob(f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{tuning_job_id}")

print(response)
# Example response:
# <vertexai.tuning._supervised_tuning.SupervisedTuningJob object at 0x7cc4bb20baf0>
# resource name: projects/1234567890/locations/us-central1/tuningJobs/4982013113894174720

resource name: projects/69210017139/locations/us-central1/tuningJobs/7223697540862967808


Testing AI model with Prompts

In [ ]:
from vertexai.generative_models import GenerativeModel

sft_tuning_job = sft.SupervisedTuningJob(f"projects/{PROJECT_ID}/locations/{REGION}/tuningJobs/{tuning_job_id}")
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)

content = '''
{
  "systemInstruction": {
    "role": "system",
    "parts": [
        {
            "text": "You are a Classifier responsible to classify into CTS,CTS Campus,Classroom Support,Print Team (External)."
        }
    ]
  },
  {
    "contents": [
        {
            "role": "USER",
            "parts": {
                "text" : "Projector in mcnair is not working."
            },
  {
    "contents": [
        {
            "role": "USER",
            "parts": {
                "text" : ""
            }
        },
    ],
    "generation_config": {
        "temperature":0.2
    }
}
'''

print(tuned_model.generate_content(content))

candidates {
  content {
    role: "model"
    parts {
      text: "Classroom Support"
    }
  }
  finish_reason: STOP
  avg_logprobs: -1.0105397701263428
}
usage_metadata {
  prompt_token_count: 172
  candidates_token_count: 2
  total_token_count: 174
  prompt_tokens_details {
    modality: TEXT
    token_count: 172
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 2
  }
}
create_time {
  seconds: 1747182116
  nanos: 105198000
}
response_id: "JOIjaO61Br6o1dkPioDXiAM"

